Evaluate a given model and compare multiple models. Push selected models to Neptune.

In [ ]:
import utils
import importlib
import pandas as pd
import numpy as np
import datetime as dt
import torch

from matplotlib.ticker import MultipleLocator
from matplotlib.dates import DayLocator, AutoDateLocator, ConciseDateFormatter
%matplotlib inline

ARCHS_DIR = 'archs'
DATA_DIR = 'data'
EXPERIMENTS_DIR = 'experiments'
DEVICE='cpu'

### Load single model for evaluation

In [ ]:
experiment_id = '0001_test'
checkpoint = 'latest-e100.pt'

model, cp = utils.load_model(experiment_id, checkpoint)

#### Plot loss and acc

In [ ]:
df_loss = pd.DataFrame({
    'trn_loss': cp['trn_losses'],
    'val_loss': cp['val_losses']
})

df_acc = pd.DataFrame({
    'trn_acc': cp.get('trn_acc', np.zeros((cp['config']['NUM_EPOCHS']))),
    'val_acc': cp.get('val_acc', np.zeros((cp['config']['NUM_EPOCHS'])))
})

# smoothen
df_loss['trn_loss'] = df_loss['trn_loss'].rolling(3, min_periods=1, center=True).mean()
df_loss['val_loss'] = df_loss['val_loss'].rolling(3, min_periods=1, center=True).mean()
df_acc['trn_acc'] = df_acc['trn_acc'].rolling(3, min_periods=1, center=True).mean()
df_acc['val_acc'] = df_acc['val_acc'].rolling(3, min_periods=1, center=True).mean()

_ = df_loss.plot(
    y=['trn_loss', 'val_loss'],
    title='Loss per epoch',
    subplots=False,
    figsize=(5,5),
    sharex=False,
    logy=True
)
_ = df_acc.plot(
    y=['trn_acc', 'val_acc'],
    title='Acc per epoch',
    subplots=False,
    figsize=(5,5),
    sharex=False,
    logy=False
)

### Load data

In [ ]:
cols = ['location', 'date', 'total_cases', 'new_cases', 'total_deaths', 'new_deaths', 'population']
dates = ['date']
df = pd.read_csv(DATA_DIR + "/" + cp['config']['DS']['SRC'],
                 usecols=cols,
                 parse_dates=dates)
df.sample()

### Backtest util

In [ ]:
def backtest(model, country='India', plot=True):
    c = country
    pop_fct = df.loc[df.location==c, 'population'].iloc[0] / 1000

    IP_SEQ_LEN = cp['config']['DS']['IP_SEQ_LEN']
    OP_SEQ_LEN = cp['config']['DS']['OP_SEQ_LEN']

    all_preds = []
    pred_vals = []
    real_vals = []

    test_data = np.array(df.loc[(df.location==c) & (df.total_cases>=100), cp['config']['DS']['FEATURES']].rolling(7, center=True, min_periods=1).mean() / pop_fct, dtype=np.float32)

    ip_shp = IP_SEQ_LEN, len(cp['config']['IP_FEATURES'])
    op_shp = OP_SEQ_LEN, len(cp['config']['OP_FEATURES'])

    for i in range(len(test_data) - IP_SEQ_LEN - OP_SEQ_LEN + 1):
        ip = torch.tensor(test_data[i : i+IP_SEQ_LEN, cp['config']['IP_FEATURES']])
        op = torch.tensor(test_data[i+IP_SEQ_LEN : i+IP_SEQ_LEN+OP_SEQ_LEN, cp['config']['OP_FEATURES']])
        ip = ip.to(DEVICE)
        op = op.to(DEVICE)

        pred = model.predict(ip.view(1, IP_SEQ_LEN, len(cp['config']['IP_FEATURES'])))    

        all_preds.append(pred.view(op_shp).cpu().numpy() * pop_fct)
        pred_vals.append(pred.view(op_shp).cpu().numpy()[0] * pop_fct)
        real_vals.append(op.view(op_shp).cpu().numpy()[0] * pop_fct)

    # prepend first input
    nans = np.ndarray((IP_SEQ_LEN, len(cp['config']['OP_FEATURES'])))
    nans.fill(np.NaN)
    pred_vals = list(nans) + pred_vals
    real_vals = list(test_data[:IP_SEQ_LEN, cp['config']['OP_FEATURES']] * pop_fct) + real_vals
    # append last N-1 values
    nans = np.ndarray(op_shp)
    nans.fill(np.NaN)
    pred_vals.extend(nans[1:]) # pad with NaN
    real_vals.extend(op.view(op_shp).cpu().numpy()[1:] * pop_fct)

    real_vals = np.array(real_vals).reshape(len(test_data), len(cp['config']['OP_FEATURES']))
    pred_vals = np.array(pred_vals).reshape(len(test_data), len(cp['config']['OP_FEATURES']))

    accs = []
    for o in range(len(cp['config']['OP_FEATURES'])):
        cmp_df = pd.DataFrame({
            'actual': real_vals[:, o],
            'predicted0': pred_vals[:, o]
        })
        # set date
        start_date = df.loc[(df.location==c) & (df.total_cases>=100)]['date'].iloc[0]
        end_date = start_date + dt.timedelta(days=cmp_df.index[-1])
        cmp_df['Date'] = pd.Series([start_date + dt.timedelta(days=i) for i in range(len(cmp_df))])

        # plot noodles
        ax=None
        i=IP_SEQ_LEN
        mape=[]
        for pred in all_preds:
            cmp_df['predicted_cases'] = np.NaN
            cmp_df.loc[i:i+OP_SEQ_LEN-1, 'predicted_cases'] = pred[:, o]
            ape = 100 * ((cmp_df['actual'] - cmp_df['predicted_cases']).abs() / cmp_df['actual'])
            mape.append(ape.mean())
            if plot: ax = cmp_df.plot(x='Date', y='predicted_cases', ax=ax, legend=False)
            i+=1
        acc = 100 - sum(mape)/len(mape)
        accs.append(acc)
        acc_str = f"{acc:0.2f}%"

        # plot primary lines
        if plot:
            ax = cmp_df.plot(
                x='Date',
                y=['actual', 'predicted0'],
                figsize=(20,8),
                lw=5,
                title=c + ' | Daily predictions | ' + acc_str,
                ax=ax
            )
            mn_l = DayLocator()
            ax.xaxis.set_minor_locator(mn_l)
            mj_l = AutoDateLocator()
            mj_f = ConciseDateFormatter(mj_l, show_offset=False)
            ax.xaxis.set_major_formatter(mj_f)
    return accs

#### Backtest one model

In [ ]:
backtest(model)

#### Backtest all models in experiment

In [ ]:
accs = []
for e in range(0, 101, 100): # start, stop, step
    checkpoint = 'latest-e' + str(e) + '.pt'
    try:
        model, cp = utils.load_model(experiment_id, checkpoint, v=False)
        acc = backtest(model, plot=False)
        accs.append(acc)
        print(checkpoint, acc)
    except Exception as e:
        print(checkpoint, e)

In [ ]:
feature = 0 # which output feature's acc should be plotted

df_exp = pd.DataFrame({
    'acc': np.array(accs)[:, feature],
    'epochs': np.arange(0, 101, 100)
})
_ = df_exp[1:].plot(
    x='epochs',
    y=['acc'],
    title='Test accuracy',
    subplots=False,
    figsize=(5,5),
    sharex=False
)

print("Models with best test accuracy:")
print(df_exp.sort_values('acc', ascending=False).head())